# SQL generation

The purpose of this notebook is to generate all of the SQL that will be needed to create and join geographic tables in postgres

In [1]:
ei_table_dict = {
    # source_table: {input args}
    'Landmarks': {
        'new_table': 'landmarks',
        'long_name': 'long',
        'lat_name': 'lat',
        'origin_srid': '4326'
    },
    'Light_Rail_Map': {
        'new_table': 'light_rail',
        'long_name': 'Longitude',
        'lat_name': 'Latitude',
        'origin_srid': '4326'
    },
    'Parks_Map': {
        'new_table': 'parks',
        'long_name': 'Longitude',
        'lat_name': 'Latitude',
        'origin_srid': '4326'
    },
    'Private_Schools': {
        'new_table': 'priv_school',
        'long_name': 'long',
        'lat_name': 'lat',
        'origin_srid': '4326'
    },
    'Public_Schools': {
        'new_table': 'pub_school',
        'long_name': 'long',
        'lat_name': 'lat',
        'origin_srid': '4326'
    }
}
# The materialized views with the counts within different buffers for each pin are named the same as the new
# table except that have '_counts' appended, e.g. 'parks_counts'

In [ ]:
# Geographically "Enable" all of the open data tables with lat long info

def create_ei_table(arg_dict):
    """
    This function generates a sql string to create table where all of the geographic information has been set up.
    
    Source table is assumed to be in the 'extra_info' schema.
    """
    def get_query(new_table, source_table, long_name, lat_name, origin_srid):
        return (
            """
DROP TABLE IF EXISTS project."{0}" CASCADE;
CREATE TABLE project."{0}" AS
    SELECT
    "{1}"."{2}" AS long,
    "{1}"."{3}" AS lat,
    ST_Transform(ST_SetSRID(ST_MakePoint("{1}"."{2}", "{1}"."{3}"), {4}), 2926) AS geom
FROM extra_info."{1}";
ALTER TABLE project."{0}" ADD gid INTEGER UNIQUE;
            """.format(new_table, source_table, long_name, lat_name, origin_srid)
        )
    
    # The base string to which everything will eventuall be appended
    core = ''
    for source_table, kwargs in arg_dict.items():
        core += get_query(source_table=source_table, **kwargs)
    return core

In [ ]:
print(create_ei_table(ei_table_dict))

In [4]:
def join_in_ei(arg_dict):
    """
    This function generates a sql string to create a table with PIN, and a count within three buffer radii.
    Serves as a lookup table for the number of counts for each extra info table.
    
    Source table is assumed to be in the 'extra_info' schema.
    """
    def get_query(new_table, source_table, dist_1, dist_2, dist_3):
        source_table = new_table  # yes, this is confusing
        new_table += '_counts'
        return (
            """
DROP MATERIALIZED VIEW IF EXISTS project."{0}";
CREATE MATERIALIZED VIEW project."{0}" AS
SELECT
everyone.ping AS pin,
everyone."count_{2}" AS "{0}{2}",
everyone."count_{3}" AS "{0}{3}",
everyone."count_{4}" AS "{0}{4}"
FROM (
(
    SELECT
    res.pin AS ping,
    count(ei.geom) AS "count_{4}"
    FROM project.geo_residential AS res LEFT JOIN project."{1}" AS ei
    ON ST_Intersects(ei.geom, res."buffer_{4}")
    GROUP BY res.pin
) AS big LEFT JOIN
(
    SELECT
    res.pin,
    count(ei.geom) AS "count_{3}"
    FROM project.geo_residential AS res LEFT JOIN project."{1}" AS ei
    ON ST_Intersects(ei.geom, res."buffer_{3}")
    GROUP BY res.pin
) AS middle ON big.ping = middle.pin LEFT JOIN
(
    SELECT
    res.pin,
    count(ei.geom) AS "count_{2}"
    FROM project.geo_residential AS res LEFT JOIN project."{1}" AS ei
    ON ST_Intersects(ei.geom, res."buffer_{2}")
    GROUP BY res.pin
) AS little ON big.ping = little.pin
) AS everyone;
            """.format(new_table, source_table, dist_1, dist_2, dist_3)
        )
    
    # The base string to which everything will eventuall be appended
    core = ''
    for source_table, kwargs in arg_dict.items():
        new_table = kwargs['new_table']
        core += get_query(new_table, source_table, '1000', '5000', '20000')
    return core

In [5]:
print(join_in_ei(ei_table_dict))


DROP MATERIALIZED VIEW IF EXISTS project."pub_school_counts";
CREATE MATERIALIZED VIEW project."pub_school_counts" AS
SELECT
everyone.ping AS pin,
everyone."count_1000" AS "pub_school_counts1000",
everyone."count_5000" AS "pub_school_counts5000",
everyone."count_20000" AS "pub_school_counts20000"
FROM (
(
    SELECT
    res.pin AS ping,
    count(ei.geom) AS "count_20000"
    FROM project.geo_residential AS res LEFT JOIN project."pub_school" AS ei
    ON ST_Intersects(ei.geom, res."buffer_20000")
    GROUP BY res.pin
) AS big LEFT JOIN
(
    SELECT
    res.pin,
    count(ei.geom) AS "count_5000"
    FROM project.geo_residential AS res LEFT JOIN project."pub_school" AS ei
    ON ST_Intersects(ei.geom, res."buffer_5000")
    GROUP BY res.pin
) AS middle ON big.ping = middle.pin LEFT JOIN
(
    SELECT
    res.pin,
    count(ei.geom) AS "count_1000"
    FROM project.geo_residential AS res LEFT JOIN project."pub_school" AS ei
    ON ST_Intersects(ei.geom, res."buffer_1000")
    GROUP BY re